In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
import cPickle as pickle
import html5lib
import os
import ipyparallel as ipp

from bs4 import BeautifulSoup as bs
from urllib2 import urlopen
from urllib2 import Request
from urllib2 import URLError

reload(sys)
sys.setdefaultencoding('utf-8')
sys.setrecursionlimit(50000)

%matplotlib inline

In [2]:
#Corremos el cluster
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

# Cargamos los datos de las propiedades

In [3]:
data = pickle.load(open("../../../../datos/DataFrame.p","rb"))

# Vamos a hacer un analisis para ver que criterio tomo para bajar un par de datos mas

In [5]:
data.info()

"""
rooms               61200
ambientes           49081
banios              13639
webp                20196
"""

'\nrooms               61200\nambientes           49081\nbanios              13639\nwebp                20196\n'

Los que menos tienen son ambientes y banios, voy a ver cuantos features no tienen ninguno de los 2

In [6]:
parc = data[pd.isnull(data['ambientes']) == True]
parc = parc[pd.isnull(parc['banios']) == True]
parc = parc[pd.isnull(parc['webp']) == True]

In [7]:
parc.info() #Tenemos 23000 masomenos

Pruebo ahora con los que no tienen banios ni cuartos

In [8]:
parc1 = data[pd.isnull(data['rooms']) == True]
parc1 = parc1[pd.isnull(parc1['banios']) == True]
parc1 = parc1[pd.isnull(parc1['webp']) == True]

In [9]:
parc1.info()

# Voy a bajar los que no tienen ni ambientes ni banios, o cuartos ni banios

In [19]:
data.head(5)

,id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,expenses,properati_url,description,title,image_thumbnail,latlon,rooms,ambientes,banios,webp
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,2012-09-19,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,NaN,http://www.properati.com.ar/udh_venta_casa_mer...,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,https://thumbs4.properati.com/1/ufoyqwscVk8n3e...,"(-34.66692, -58.70097)",NaN,NaN,2.0,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...
1,a205c53378e7879cf7519659317fb410c060d20e,2012-10-22,sell,house,San Antonio De Padua,|Argentina|Bs.As. G.B.A. Zona Oeste|Merlo|San ...,Argentina,Bs.As. G.B.A. Zona Oeste,3429244.0,NaN,...,NaN,http://www.properati.com.ar/20hi_venta_casa_me...,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,https://thumbs4.properati.com/5/I5RS_jAIUseo9H...,"(-34.66692, -58.70097)",3.0,4.0,NaN,None
2,9446052c949050359fb08b9f1ba83ba2eaa20c99,2012-12-08,sell,house,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6544492346,-58.4895603251",...,NaN,http://www.properati.com.ar/4m4m_venta_casa_ma...,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,https://thumbs4.properati.com/6/DsNJjCb5_zIPV0...,"(-34.6544492346, -58.4895603251)",5.0,3.0,NaN,None
3,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,2013-01-04,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.429071,-58.795418",...,NaN,http://www.properati.com.ar/518a_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,https://thumbs4.properati.com/8/bJn9FqcBRGi_8H...,"(-34.429071, -58.795418)",2.0,2.0,NaN,None
4,61f3789618edabeaa7b5fc06ff36112f8a06557f,2013-01-26,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.439241,-58.798589",...,NaN,http://www.properati.com.ar/59ec_venta_departa...,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,https://thumbs4.properati.com/5/hiXoFGFHBiIhV2...,"(-34.439241, -58.798589)",2.0,NaN,NaN,None


In [4]:
dataR = data[['id','properati_url','rooms','ambientes','banios','webp']]

In [5]:
dataR = dataR[5000:10000]

In [6]:
#data.head(5)

In [7]:
#data.info()

In [9]:
def chequearURL(x):
    
    if( pd.isnull(x[1]) ): return None
    if( pd.isnull(x[2]) and pd.isnull(x[4]) ): return buscarURL(x[1])
    if( pd.isnull(x[3]) and pd.isnull(x[4]) ): return buscarURL(x[1])
    
    return None
        
def buscarURL(x):
    
    req = Request(x)
    try:
        resp = urlopen(req, timeout=10)
    except URLError:
        print("Bad URL or timeout\n")
        return None
    except requests.ConnectionError as e:
        print("conexion perdida\n")
        return None
    except requests.Timeout as e:
        print("Timeout Error\n")
        return None
    except requests.RequestException as e:
        print("General Error\n")
        return None
    except KeyboardInterrupt:
        print("Someone closed the program\n")

    resp_page = resp.read()
    
    return resp_page

In [ ]:
dataR['webp'] = dataR.apply(chequearURL, axis=1)

In [ ]:
pickle.dump(dataR,open("save2.p","wb"))